In [ ]:
import tensorflow as tf
import re
from functools import partial

In [ ]:
print(tf.__version__)

2.15.0


In [ ]:
!python --version

Python 3.10.12


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
import os
from google.colab import drive
from google.colab import files

In [ ]:
MOUNTPOINT = '/content/gdrive'
DATADIR = os.path.join(MOUNTPOINT, 'My Drive', 'Learn TensorFlow', 'Data')
drive.mount(MOUNTPOINT)

Mounted at /content/gdrive


In [ ]:
path = os.path.join(DATADIR, '2022_twitter_data.csv')
loaded_data = pd.read_csv(path)
loaded_data

,content,hashtags,UserDescription
0,I’m getting a sugar daddy this year!,NaN,NCAT Alumna |🪬 | | For giggles
1,might make em french toast after and spoon a l...,NaN,"writer, reader, horror lover, general disaster..."
2,Our platform can be matched to your company’s ...,NaN,An innovative way to train and learn. \nSign u...
3,Oh my God this first song. I can't with how am...,NaN,Vegan activist trying to make a change in the ...
4,Happy new year ! 😚🎉,NaN,5♥5♥5
...,...,...,...
547479,ECG be the source 😂✌🏽,NaN,Jesus 💜 | ART PR @artblockgh1 ||\n🎪 @DBkyeiEve...
547480,i’m hungry,NaN,element 85 astatine
547481,Playing bad bunny all shift bc customers are b...,NaN,bcn | 20
547482,this is me btw. when you're mean to me this is...,NaN,"aspiring dude, accomplished guy, and occasiona..."


In [ ]:
data = loaded_data

In [ ]:
data = data.drop(columns=['hashtags', 'UserDescription'])
data

,content
0,I’m getting a sugar daddy this year!
1,might make em french toast after and spoon a l...
2,Our platform can be matched to your company’s ...
3,Oh my God this first song. I can't with how am...
4,Happy new year ! 😚🎉
...,...
547479,ECG be the source 😂✌🏽
547480,i’m hungry
547481,Playing bad bunny all shift bc customers are b...
547482,this is me btw. when you're mean to me this is...


In [ ]:
data['content'].isnull().sum()

0

In [ ]:
def clean(text):
    text = str(text).lower()
    text = re.sub(r'https?:\/\/\S+', '', text)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    return text

data['content'] = data['content'].apply(clean)
data

,content
0,im getting a sugar daddy this year
1,might make em french toast after and spoon a l...
2,our platform can be matched to your companys b...
3,oh my god this first song i cant with how amaz...
4,happy new year
...,...
547479,ecg be the source
547480,im hungry
547481,playing bad bunny all shift bc customers are b...
547482,this is me btw when youre mean to me this is w...


In [ ]:
data.to_csv('cleaned_2022_twitter_data.csv')

In [ ]:
DATADIR = os.path.join(MOUNTPOINT, 'My Drive', 'Learn TensorFlow', 'Models')
drive.mount(MOUNTPOINT)
path = os.path.join(DATADIR, 'sentiment_analysis_model.keras')
model = tf.keras.models.load_model(path)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
model.save('sentiment_analysis_model_tf', save_format='tf')

In [ ]:
%cd /content/
!zip -r sentiment_analysis_model_tf.zip sentiment_analysis_model_tf

/content
  adding: sentiment_analysis_model_tf/ (stored 0%)
  adding: sentiment_analysis_model_tf/saved_model.pb (deflated 87%)
  adding: sentiment_analysis_model_tf/keras_metadata.pb (deflated 87%)
  adding: sentiment_analysis_model_tf/fingerprint.pb (stored 0%)
  adding: sentiment_analysis_model_tf/assets/ (stored 0%)
  adding: sentiment_analysis_model_tf/variables/ (stored 0%)
  adding: sentiment_analysis_model_tf/variables/variables.data-00000-of-00001 (deflated 8%)
  adding: sentiment_analysis_model_tf/variables/variables.index (deflated 58%)


In [24]:
search = 'climate change'
search_data = pd.DataFrame()
search_data = data[data['content'].str.contains(search)]
search_data.columns = ['text_vectorization_input']
model_input = tf.data.Dataset.from_tensor_slices(search_data.to_dict(orient="list"))
analysis = model.predict(model_input.batch(32))

num_ones = 0
num_zeros = 0

for prediction in analysis:
    if prediction[0] >= 0.5:
        prediction[0] = 1
        num_ones += 1
    else:
        prediction[0] = 0
        num_zeros += 1

percent_zeros = num_zeros / (num_ones + num_zeros)

6/6 [==============================] - 0s 49ms/step


In [ ]:
percent_zeros

0.3101391650099404

In [26]:
climate_change_data = pd.DataFrame()
climate_change_data = data[data['content'].str.contains('climate change')]
climate_change_data

,content
293,the christmas lyric im dreaming of a white chr...
2402,oh\n\ni had no idea the people had power over ...
4540,the reason for that is simple manchin and schu...
6185,double my salary end supremacy end homelessnes...
14008,ok so trains are expensive got it heres the p...
...,...
536267,the double impact of climate change and conflict
540310,tbch idk if hes just trying to pissed us off i...
543163,just saw this on the news\n\nive always known ...
544362,our cookie collab with renewalmill is now an i...


In [27]:
climate_change_data.empty

False

In [28]:
X_data_df = climate_change_data
X_data = tf.data.Dataset.from_tensor_slices(X_data_df.to_dict(orient="list"))

In [29]:
X_data_df.columns = ['text_vectorization_input']
X_data_df

,text_vectorization_input
293,the christmas lyric im dreaming of a white chr...
2402,oh\n\ni had no idea the people had power over ...
4540,the reason for that is simple manchin and schu...
6185,double my salary end supremacy end homelessnes...
14008,ok so trains are expensive got it heres the p...
...,...
536267,the double impact of climate change and conflict
540310,tbch idk if hes just trying to pissed us off i...
543163,just saw this on the news\n\nive always known ...
544362,our cookie collab with renewalmill is now an i...


In [30]:
X_data = tf.data.Dataset.from_tensor_slices(X_data_df.to_dict(orient="list"))

In [33]:
y_proba = model.predict(X_data.batch(32))

6/6 [==============================] - 0s 44ms/step


In [34]:
y_proba

array([[0.6314597 ],
       [0.27208784],
       [0.24288651],
       [0.91516215],
       [0.28090915],
       [0.98341006],
       [0.3055187 ],
       [0.35229728],
       [0.11240219],
       [0.2879988 ],
       [0.75999945],
       [0.9480067 ],
       [0.9649219 ],
       [0.635834  ],
       [0.22547129],
       [0.01718277],
       [0.13417171],
       [0.13395505],
       [0.63211197],
       [0.65778184],
       [0.5175939 ],
       [0.34460697],
       [0.10209019],
       [0.8180669 ],
       [0.18549329],
       [0.5564367 ],
       [0.37854102],
       [0.1307789 ],
       [0.30905592],
       [0.8527921 ],
       [0.29982752],
       [0.7409882 ],
       [0.2904239 ],
       [0.6626141 ],
       [0.6687329 ],
       [0.2851793 ],
       [0.32493988],
       [0.43139696],
       [0.8793812 ],
       [0.5105229 ],
       [0.7678897 ],
       [0.71721154],
       [0.5204969 ],
       [0.9372666 ],
       [0.37591648],
       [0.28017035],
       [0.17561205],
       [0.427

In [35]:
y_pred = y_proba.copy()
num_ones = 0
num_zeros = 0

for prediction in y_pred:
  if prediction[0] >= 0.5:
    prediction[0] = 1
    num_ones += 1
  else:
    prediction[0] = 0
    num_zeros += 1

print(y_pred)
print(num_ones)
print(num_zeros)

[[1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]
 [1.]
 [1.]
 [0.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.

In [36]:
print("Percent zeros", num_zeros / (num_ones + num_zeros))

Percent zeros 0.49171270718232046
